In [78]:

import tensorflow as tf



from kaggle_datasets import KaggleDatasets

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import os
import time
from IPython import display
from PIL import Image
import numpy as np

from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

#'kaggle datasets download -d spandan2/cats-faces-64x64-for-generative-models'
BUFFER_SIZE = 1193
BATCH_SIZE = 64



In [79]:
x_train = np.load('../input/monet-to-numpy/monet.npy')

In [80]:

print(x_train.shape)
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(1)

In [81]:
plt.imshow(x_train[0]*0.5 +0.5)

In [82]:
def build_generator3(channels = 3):
    GENERATE_RES = 3
    model = Sequential()
    N = 512
    N_DIM = 8
    model.add(Dense(N_DIM*N_DIM*N,activation="relu",input_shape=(100,)))
    model.add(Reshape((N_DIM,N_DIM,N)))

    model.add(UpSampling2D())
    model.add(Conv2D(N,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(N/2,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    model.add(UpSampling2D())
    model.add(Conv2D(N/4,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(N/8,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))



    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    return model
generator =build_generator3()
generator.summary()



In [83]:

def build_discriminator3( image_shape = [128, 128, 3]):
    model = Sequential()
    N = 512
    model.add(Conv2D(N/16, kernel_size=3, strides=2, input_shape=image_shape, 
                     padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(N/8, kernel_size=3, strides=2, padding="same"))
    
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(N/4, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(N/2, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

  

    model.add(Dropout(0.25))
    model.add(Conv2D(N, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model
discriminator =build_discriminator3()
discriminator.summary()




In [84]:
4*524288

In [85]:
#generator = build_generator2()

#generator, layers_gen = auto_build_generator(128)
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training = False)


plt.imshow(generated_image[0]*0.5 +0.5)

In [86]:


cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [87]:
def discriminator_loss(real_output, fake_output):
    coeff = 1
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss*coeff

In [88]:
def generator_loss(fake_output):
    coeff = 1
    return cross_entropy(tf.ones_like(fake_output)*coeff, fake_output*coeff)


generator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)

In [89]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

EPOCHS = 500
noise_dim = 100
num_examples_to_generate = 16
num_trie = 1
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate*num_trie, noise_dim])

In [90]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss
    

In [91]:
def get_best_pred(gen, disc, seeds, num):
    
    num_iter = num
    best_pred = []
    for i in range(16):
        ite = i*num_iter
        s = seeds[ite:ite+num_iter]
        
        preds = gen(s, training=False)
        
        preds_disc = disc(preds)
        best = np.argmax(preds_disc)
        best_pred.append(preds[best])
    return np.array(best_pred)


def show_best(gen, disc):
    samples = 16
    among = 2
    test_input = tf.random.normal([samples*among, noise_dim])
    predictions = get_best_pred(gen, disc, test_input, among)
    pred_disc = disc(predictions, training=False)
    fig = plt.figure(figsize=(32,32))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i]*0.5 +0.4 )
        plt.axis('off')
        
    plt.show()
    print(np.array(pred_disc).reshape(4,4))
    print(predictions.shape[0])
    print(np.min(predictions[i]))

In [92]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        G_loss = []
        D_loss = []
        for i,image_batch in enumerate(dataset):
            #print('iteration : ', i)
            g_loss, d_loss = train_step(image_batch)
            G_loss.append(g_loss)
            D_loss.append(d_loss)
        
        G_loss = np.mean(G_loss)
        D_loss = np.mean(D_loss)
        
        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate_and_save_images(generator, discriminator,epoch + 1,seed)
                                 
                                 
                                 
    
        # Save the model every 15 epochs
        #checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec... G_loss = {} , D_loss = {}'.format(epoch + 1, time.time()-start, G_loss, D_loss))

      # Generate after the final epoch
    #display.clear_output(wait=True)
    generate_and_save_images(generator,
                            discriminator,
                            epochs,
                            seed)

In [93]:
def generate_and_save_images(model, disc, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    
    predictions = model(test_input, training=False)
    pred_disc = disc(predictions)
    
    #predictions = get_best_pred(model, disc, test_input)
    #pred_disc = disc(predictions)
    
    fig = plt.figure(figsize=(8,8))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i]*0.5 +0.5)
        plt.axis('off')

    #plt.savefig('out/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    #print(pred_disc)
    print(np.array(pred_disc).reshape(4,4))
    

In [94]:
essaie = '2'
#generator.load_weights('../input/monetgan/generatorMONET1.h5')
#discriminator.load_weights('../input/monetgan/discriminatorMONET1.h5')

In [ ]:

essaie = '2'


for i in range(10):
  print('epoch i = ', i*10)
  show_best(generator, discriminator)
  train(train_dataset, 10)
  print('epoch i = ', i)

  generator.save_weights('generatorMONET1.h5')
  discriminator.save_weights('discriminatorMONET1.h5')

In [ ]:
generator.save_weights('/content/drive/My Drive/GAN_MONET/generatorMONET'+ str(essaie) +'.h5')
discriminator.save_weights('/content/drive/My Drive/GAN_MONET/discriminatorMONET'+ str(essaie) +'.h5')

In [ ]:
  generator.save_weights('generatorMONET1.h5')
  discriminator.save_weights('discriminatorMONET1.h5')

In [ ]:
nois = tf.random.normal([1, noise_dim])
imag = generator(nois, training=False)

out = discriminator(imag, training = False)
print(out)
it = iter(test)
float(discriminator_loss(next(it), out))